In [1]:
import pandas as pd
import numpy as np
import pandas_ta as ta 
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
# Function definitions
def wavetrend(df):
    n1 = 9  # channel length
    n2 = 12  # average length
    df['ap'] = (df.High + df.Low + df.Close) / 3  # WT ma source
    df['esa'] = ta.ema(df.ap, n1)  # EMA of the ma source by the channel length
    df['d'] = ta.ema(abs(df.ap - df.esa), n1)
    df['ci'] = (df.ap - df.esa) / (0.015 * df.d)
    df['wt1'] = ta.ema(df.ci, n2)
    df['wt2'] = ta.sma(df.wt1, 3)
    return df

In [3]:
def cross_signal(df):
    df['cross'] = np.select([(df['wt1'].shift(1) < df['wt2'].shift(1)) & (df['wt1'] > df['wt2']),
                             (df['wt1'].shift(1) > df['wt2'].shift(1)) & (df['wt1'] < df['wt2'])],
                            ['up', 'down'])

    df['signal'] = np.select([(df.mfrsi > 0) &
                              (df.Close > df.ma200) &
                              (df.LL < df.ma50) &
                              (df.wt1 < 0) &
                              (df.wt2 < 0) &
                              (df.cross == 'up'),
                              (df.mfrsi < 0) &
                              (df.Close < df.ma200) &
                              (df.HH > df.ma50) &
                              (df.wt1 > 0) &
                              (df.wt2 > 0) &
                              (df.cross == 'down')],
                             [1, 2])
    return df

In [4]:
def execute(df):
    
    trades = []
    entry = 0
    exit = 0
    stop = 0
    profit = 0
    entry_date = 0
    exit_date = 0
    position = None

    for i in range(len(df)):
        if position == None:

            # long entry
            if df.signal.iloc[i] == 1:
                entry = df.Close.iloc[i]
                stop = df.LL.iloc[i]
                profit = (entry - stop) * 2 + entry
                entry_date = df.index[i]
                position = 'Long'

            # short entry
            elif df.signal.iloc[i] == 2:
                entry = df.Close.iloc[i]
                stop = df.HH.iloc[i]
                profit = entry - (stop - entry) * 2
                entry_date = df.index[i]
                position = 'Short'

       # long exit
        if position == 'Long':
            if df.High.iloc[i] > profit and df.Open.iloc[i] < profit:
                exit = profit
                exit_date = df.index[i]
            elif df.Open.iloc[i] > profit:
                exit = df.Open.iloc[i]
                exit_date = df.index[i]
            elif df.Low.iloc[i] <= stop and df.Open.iloc[i] > stop:
                exit = stop
                exit_date = df.index[i]
            elif df.Open.iloc[i] < stop:
                exit = df.Open.iloc[i]
                exit_date = df.index[i]

        # short exit
        if position == 'Short':
            if df.Low.iloc[i] < profit and df.Open.iloc[i] > profit:
                exit = profit
                exit_date = df.index[i]
            elif df.Open.iloc[i] < profit:
                exit = df.Open.iloc[i]
                exit_date = df.index[i]
            elif df.High.iloc[i] >= stop and df.Open.iloc[i] < stop:
                exit = stop
                exit_date = df.index[i]
            elif df.Open.iloc[i] > stop:
                exit = df.Open.iloc[i]
                exit_date = df.index[i]

        # record the trades
        if exit !=0:
            if position == 'Long':
                trades.append({'Entry Date':entry_date,
                             'Direction': position,
                             'Entry Price': entry,
                             'Profit': profit,
                             'Stop': stop,
                             'Exit Price': exit,
                             'Exit Date':exit_date,
                             'PL':exit - entry})

            elif position == 'Short':
                trades.append({'Entry Date':entry_date,
                             'Direction': position,
                             'Entry Price': entry,
                             'Profit': profit,
                             'Stop': stop,
                             'Exit Price': exit,
                             'Exit Date':exit_date,
                             'PL':entry - exit})

            entry_date = 0
            position = None
            entry = 0
            profit = 0
            stop = 0
            exit = 0
            exit_date = 0

    trades_df = pd.DataFrame(trades)    
    return trades_df

spreadsheets

In [5]:
# file_paths = {
#     "AUDUSD": "C:/Users/your_file_path/AUD.csv",
#     "EURUSD": "C:/Users/your_file_path/EUR.csv",
#     "USDJPY": "C:/Users/your_file_path/JPY.csv",
#     "GBPUSD": "C:/Users/your_file_path/GBP.csv",
#     "ES": "C:/Users/your_file_path/ES.csv",
#     "NQ": "C:/Users/your_file_path/NQ.csv",
#     "DAX": "C:/Users/your_file_path/DAX.csv",
#     "NIKKEI": "C:/Users/your_file_path/NIKKEI.csv",
#     "GOLD": "C:/Users/your_file_path/GOLD.csv",
#     "SILVER": "C:/Users/your_file_path/SILVER.csv",
#     "OIL": "C:/Users/your_file_path/OIL.csv",
#     "BTC": "C:/Users/your_file_path/BTC.csv",
#     "ETH": "C:/Users/your_file_path/ETH.csv",
#     "TEN YEAR NOTES": "C:/Users/your_file_path/TY.csv"
# }


In [6]:
# all_trades = []

# # Process each file
# for code, file_path in file_paths.items():
#     df = pd.read_csv(file_path)
    
#     # Ensure Date column is correctly parsed and set as index
#     df['Date'] = pd.to_datetime(df['Date'], unit='s')
#     df.set_index('Date', inplace=True)
    
#     # Calculate technical indicators
#     df = wavetrend(df)
#     period, multi, posy = 60, 150, 2.5
#     df['mfrsi'] = (ta.sma(((df.Close - df.Open) / (df.High - df.Low) * multi), period)) - posy
#     df['ma50'] = ta.ema(df.Close, 50)
#     df['ma200'] = ta.ema(df.Close, 200)
#     df['HH'] = df.High.rolling(window=5).max()
#     df['LL'] = df.Low.rolling(window=5).min()
#     df = cross_signal(df)
    
#     # Execute trades
#     trades_df = execute(df)
    
#     # Add code column to trades_df
#     trades_df['Code'] = code
    
#     # Append trades to all_trades list
#     all_trades.append(trades_df)

# # Concatenate all trades into a single DataFrame
# results_df = pd.concat(all_trades, ignore_index=True)


yfinance

In [7]:
codes = ['aapl','amzn','meta','nvda','googl','msft','tsla']

In [8]:
all_trades = []

# Process each file
for code in codes:
    # Determine the maximum date range available (730 days from today)
    end_date = datetime.today()
    start_date = end_date - timedelta(days=729)
    df = yf.download(code, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'), interval='1h')
     
    # Calculate technical indicators
    df = wavetrend(df)
    period, multi, posy = 60, 150, 2.5
    df['mfrsi'] = (ta.sma(((df.Close - df.Open) / (df.High - df.Low) * multi), period)) - posy
    df['ma50'] = ta.ema(df.Close, 50)
    df['ma200'] = ta.ema(df.Close, 200)
    df['HH'] = df.High.rolling(window=5).max()
    df['LL'] = df.Low.rolling(window=5).min()
    df = cross_signal(df)
    df = df.dropna()
    
    # Execute trades
    trades_df = execute(df)
    
    # Add code column to trades_df
    trades_df['Code'] = code
    
    # Append trades to all_trades list
    all_trades.append(trades_df)

# Concatenate all trades into a single DataFrame
results_df = pd.concat(all_trades, ignore_index=True)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
results_df

,Entry Date,Direction,Entry Price,Profit,Stop,Exit Price,Exit Date,PL,Code
0,2022-08-22 12:30:00-04:00,Long,168.660004,169.960022,168.009995,168.009995,2022-08-22 13:30:00-04:00,-0.650009,aapl
1,2022-08-24 11:30:00-04:00,Long,167.615005,170.355026,166.244995,170.570007,2022-08-26 09:30:00-04:00,2.955002,aapl
2,2022-10-05 09:30:00-04:00,Short,143.410004,137.790009,146.220001,146.220001,2022-10-05 12:30:00-04:00,-2.809998,aapl
3,2022-10-14 10:30:00-04:00,Short,140.380005,132.100006,144.520004,145.490005,2022-10-18 09:30:00-04:00,-5.110001,aapl
4,2022-10-18 11:30:00-04:00,Short,144.350006,139.650024,146.699997,146.699997,2022-10-21 14:30:00-04:00,-2.349991,aapl
...,...,...,...,...,...,...,...,...,...
334,2024-05-24 09:30:00-04:00,Long,177.819901,186.939713,173.259995,173.259995,2024-05-28 09:30:00-04:00,-4.559906,tsla
335,2024-05-28 15:30:00-04:00,Long,176.740005,180.580002,174.820007,174.429993,2024-05-29 09:30:00-04:00,-2.310013,tsla
336,2024-05-29 12:30:00-04:00,Long,176.830002,182.090012,174.199997,182.090012,2024-05-30 09:30:00-04:00,5.260010,tsla
337,2024-05-31 15:30:00-04:00,Long,178.059998,186.539795,173.820099,173.820099,2024-06-05 09:30:00-04:00,-4.239899,tsla


In [10]:

# Function to calculate win percentage for a given DataFrame
def calculate_win_percentage(df):
    total_trades = len(df)
    winning_trades = len(df[df['PL'] > 0])
    win_percentage = (winning_trades / total_trades) * 100
    return win_percentage

# Group by 'Code' and apply the win percentage calculation
win_percentage_by_contract = results_df.groupby('Code').apply(calculate_win_percentage).reset_index()

# Rename the columns for clarity
win_percentage_by_contract.columns = ['Code', 'Win Percentage']

# Calculate the overall win percentage
total_trades = len(results_df)
total_winning_trades = len(results_df[results_df['PL'] > 0])
total_win_percentage = (total_winning_trades / total_trades) * 100

# Add the total win percentage to the DataFrame
total_row = pd.DataFrame([['Total', total_win_percentage]], columns=['Code', 'Win Percentage'])
win_percentage_by_contract = pd.concat([win_percentage_by_contract, total_row], ignore_index=True)

print(win_percentage_by_contract)


    Code  Win Percentage
0   aapl       27.659574
1   amzn       35.820896
2  googl       35.555556
3   meta       48.717949
4   msft       37.142857
5   nvda       36.842105
6   tsla       44.897959
7  Total       37.758112
